<a href="https://colab.research.google.com/github/JoshFowlkes/DS-Unit-2-Kaggle-Challenge/blob/master/module4/assignment_kaggle_challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 4

## Catch up, if needed
- [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2/portfolio-project/ds6), then choose your dataset, and [submit this form](https://forms.gle/nyWURUg65x1UTRNV9), due yesterday at 3:59pm Pacific.
- Submit predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.) The competition closes today at 3:59pm. Every student should make at least one submission that scores at least 60% accuracy (above the majority class baseline).

## Assignment
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_ The competition closes today at 3:59pm.
- [ ] Add comments and Markdown to your notebook. Clean up your code.
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Try combining xgboost early stopping, cross-validation, & hyperparameter optimization, with [the "original" (non scikit-learn) xgboost API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.cv).
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

#### Try stacking multiple submissions!

Here's some code you can use:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # eli5, version >= 0.9
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders eli5 pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module3')

     |████████████████████████████████| 92kB 3.1MB/s 
     |████████████████████████████████| 102kB 28.8MB/s 
     |████████████████████████████████| 133kB 45.6MB/s 
     |████████████████████████████████| 7.1MB 41.1MB/s 
     |████████████████████████████████| 614kB 37.2MB/s 
     |████████████████████████████████| 225kB 43.4MB/s 
     |████████████████████████████████| 768kB 35.9MB/s 
     |████████████████████████████████| 194kB 46.0MB/s 
     |████████████████████████████████| 51kB 20.4MB/s 
     |████████████████████████████████| 51kB 20.5MB/s 
     |████████████████████████████████| 737kB 44.8MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145036 sha256=b413f5eb16988e132ae60f9922c5831aa26cf1cab99498709e874ee85f83690c
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=ee1f25b12e5813fb8

In [0]:
from sklearn.model_selection import train_test_split


# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

In [0]:
# Data Cleaning and Wrangling
def wrangle(X):
    
    
    X = X.copy()
    
    # fixing the latitude outlier
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # replacing 0s with nans, so that imputer can have a field day with them
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # handling duplicates
    duplicates = ['quantity_group']
    X = X.drop(columns=duplicates)
    
    # handling unusable data
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # changing the date string to date time format
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # calculating years since construction
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # giving numerical quantities to payment type category
    X['payment_type'] = X.payment_type.replace({'annually' : 2, 'never pay' : 0,
                                                'unknown' : 0, 'on failure' : 0,'other' : 0,
                                                'per bucket' : 1, 'monthly' : 1})
    
    
    # in early 2000s Tanzanian Gov enacted National Water Policy NAWAPO
    X['new generation'] = ((X['construction_year'] > 2000).map({True : 1, False : 0}))
     
    # changing regions to numeric values for their category of gov support/funding  
    X['region'] = X.region.replace({'Iringa' : 2, 'Arusha' : 2, 'Manyara' : 0, 'Shinyanga' : 2,
                                'Mbeya' : 2, 'Kilimanjaro' : 0, 'Morogoro' : 2, 'Kagera' : 0,
                                'Mwanza' : 1, 'Kigoma' : 0, 'Ruvuma' : 0, 'Pwani' : 0, 'Tanga' : 1,
                                'Dodoma' : 1, 'Singida' : 0, 'Mara' : 0, 'Dar es Salaam' : 0,
                                'Tabora' : 1, 'Rukwa' : 0, 'Lindi' : -2, 'Mtwara' : 1})
    
    
    
    return X

train = wrangle(train)
test = wrangle(test)

In [0]:

target = 'status_group'
X_train = train.drop(columns=target)
y_train = train[target]
X_test = test

In [0]:

target = 'status_group'

train_features = train.drop(columns=[target])


numeric_features = train_features.select_dtypes(include='number').columns.tolist()

high_cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = high_cardinality[high_cardinality<=50].index.tolist()

features = numeric_features + categorical_features

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]

In [100]:
%%time
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

pipeline = make_pipeline(
      ce.OrdinalEncoder(),
      SimpleImputer(),
      StandardScaler(),
      XGBClassifier(objective = 'multi:softmax', booster = 'gbtree',
                    nrounds = 'min.error.idx', num_class = 3,
                    maximize = False, eval_metric = 'merror', eta = .2,
                    max_depth = 19, colsample_bytree = .55, nthread = -1, n_jobs = -1,
                    learning_rate=.1,  gamma = 1, n_estimators=210, random_state=69,
                    scale_pos_weight=13)


)

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_train)
print('Validation Accuracy: ', accuracy_score(y_train, y_pred))

Validation Accuracy:  0.9505050505050505
CPU times: user 4min 58s, sys: 412 ms, total: 4min 59s
Wall time: 2min 30s


# Finding best Hyper Parameter with RandomizedSearchCV

In [0]:
%%time
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

#pipeline = make_pipeline(
      #ce.OrdinalEncoder(), 
      ce.TargetEncoder(min_samples_leaf=1, smoothing=1),
      SimpleImputer(),
      StandardScaler(),
      XGBClassifier(objective = 'multi:softmax', booster = 'gbtree',
                    nrounds = 'min.error.idx', num_class = 3,
                    maximize = False, eval_metric = 'merror', 
                     colsample_bytree = .55, n_jobs = -1,
                     random_state=69)


)

#param_distributions = {
    'xgbclassifier__n_estimators': randint(50, 1000), 
    'xgbclassifier__max_depth': randint(3, 35), 
    'xgbclassifier__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
    'xgbclassifier__gamma': randint(1,10),
    'xgbclassifier__eta' : [.1,.2,.3,.4,.5,.6,.7,.8]
}

#search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=5, 
    cv=2, 
    scoring=None, 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

**This code cell takes nearly 40 minutes to run and I already ran it in another notebook, so I'll just paste the results I got here:**

```
Fitting 2 folds for each of 5 candidates, totalling 10 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 37.1min finished
CPU times: user 21min 1s, sys: 12.8 s, total: 21min 14s
Wall time: 47min 45s


Best hyperparameters {'xgbclassifier__eta': 0.4, 'xgbclassifier__gamma': 1, 'xgbclassifier__learning_rate': 0.001, 'xgbclassifier__max_depth': 16, 'xgbclassifier__n_estimators': 834}
```

In [0]:
# although weirdly, i tried parameters aside from these ones and actually got better results

# Final Submissions To Kaggle Challenge

In [0]:
y_pred = pipeline.predict(X_test)

submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)

!head my-ultimate-ensemble-submission.csv

In [0]:
if in_colab:
    from google.colab import files
     #Just try again if you get this error:
     #TypeError: Failed to fetch
     #https://github.com/googlecolab/colabtools/issues/337
    files.download('my-ultimate-ensemble-submission.csv')
         